In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%load_ext line_profiler

In [3]:
from pyk4a import (
    PyK4A,
    Config,
    ColorResolution,
    DepthMode,
    WiredSyncMode,
)

In [4]:
from multicamera_acquisition.interfaces.arduino import (
    packIntAsLong,
    wait_for_serial_confirmation,
)
import struct
import time
import numpy as np

# from multicamera_acquisition.interface g hs import get_camera
from multicamera_acquisition.video_io import write_frame
from multicamera_acquisition.paths import ensure_dir
from multicamera_acquisition.acquisition import acquire_video

from pypylon import pylon

/home/dattalab/code/multicamera_acquisition/multicamera_acquisition/interfaces/arduino.py:4: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [5]:
import logging
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

In [6]:
!k4arecorder --list

Index:0	Serial:000161621712	Color:1.6.110	Depth:1.6.79
Index:1	Serial:000412721712	Color:1.6.110	Depth:1.6.79


## Set up the camera

In [7]:
recording_duration_s = 60 #60*30  # seconds
framerate = 90
exposure_time=950# 1000s
save_location = './tmp'
quality = 15
display = False
camera_list = [
    {'name': 'top','serial': 40338090, 'brand':'basler','gain': 30, 'gpu':0, "display":True, "trigger_line":"Line2", "quality": quality, "exposure_time":exposure_time},
    {'name': 'S1', 'serial': 40338089, 'brand':'basler', 'gain': 30, 'gpu':1, "display":display, "trigger_line":"Line2", "quality": quality, "exposure_time":exposure_time},
    {'name': 'S2', 'serial': 40338086, 'brand':'basler', 'gain': 30, 'gpu':0, "display":display, "trigger_line":"Line2", "quality": quality, "exposure_time":exposure_time},
    {'name': 'S3', 'serial': 40338106, 'brand':'basler', 'gain': 30, 'gpu':1, "display":display, "trigger_line":"Line2", "quality": quality, "exposure_time":exposure_time},
    { 'name': 'S4', 'serial': 40338091, 'brand':'basler', 'gain': 30, 'gpu':0, "display":display, "trigger_line":"Line2", "quality": quality, "exposure_time":exposure_time},
    {'name': 'bottom', 'serial': 40338109, 'brand':'basler', 'gain': 30, 'gpu':1, "display":True, "trigger_line":"Line2", "quality": quality, "exposure_time": exposure_time},
    { 'name': 'azuretop', 'serial': '000412721712', 'brand':'azure', 'gpu':0, "display":display, "quality": quality},
    { 'name': 'azurebottom', 'serial': '000161621712', 'brand':'azure', 'gpu':1, "display":display, "quality": quality},
]

In [ ]:
video_out=acquire_video(
    save_location,
    camera_list,
    frame_timeout = 1000, # how long to allow frames to attempt to grab bpcie lanesefore a timeout
    azure_recording=True,
    azure_framerate = 30,  # azure's frame rate in this case.
    framerate=framerate,
    display_framerate = 30,
    recording_duration_s = recording_duration_s,
    serial_timeout_duration_s=0.1,
    overwrite=True,
    append_datetime=True,
    dropped_frame_warnings=True,
    verbose=True
)

INFO:root:Checking cameras...
INFO:root:Initializing cameras...
INFO:root:Camera top...
INFO:root:Initialized top (40338090)
INFO:root:Camera S1...
INFO:root:Initialized S1 (40338089)
INFO:root:Camera S2...
INFO:root:Initialized S2 (40338086)
INFO:root:Camera S3...
INFO:root:Initialized S3 (40338106)
INFO:root:Camera S4...
INFO:root:Initialized S4 (40338091)
INFO:root:Camera bottom...
INFO:root:Initialized bottom (40338109)
INFO:root:Camera azuretop...


Index:0	Serial:000161621712	Color:1.6.110	Depth:1.6.79
Index:1	Serial:000412721712	Color:1.6.110	Depth:1.6.79


INFO:root:Initialized azuretop (000412721712)
INFO:root:Camera azurebottom...


Index:0	Serial:000161621712	Color:1.6.110	Depth:1.6.79
Index:1	Serial:000412721712	Color:1.6.110	Depth:1.6.79


INFO:root:Initialized azurebottom (000161621712)
INFO:root:Initializing Arduino...
INFO:root:Waiting 2s to wait for arduino to connect...
INFO:root:Preparing acquisition loops
INFO:root:Telling arduino to start recording
DEBUG:root:Inverse framerate: 11111; num cycles: 1800
/home/dattalab/code/multicamera_acquisition/multicamera_acquisition/interfaces/arduino.py:32: UserWarning: PySerial: "Start" confirmation expected, got "Waiting..."". Trying again.
  warnings.warn(
/home/dattalab/code/multicamera_acquisition/multicamera_acquisition/interfaces/arduino.py:32: UserWarning: PySerial: "Start" confirmation expected, got "0"". Trying again.
  warnings.warn(
INFO:root:Starting Acquisition...


Confirmation recieved: Start


recording progress (s):   0%|                                                                                                                                                                                                                              | 0/60 [00:00<?, ?it/s]INFO:root:confirmation
INFO:root:confirmation
INFO:root:confirmation
INFO:root:confirmation
INFO:root:confirmation
INFO:root:FRAME SHAPE (1200, 1920)
INFO:root:FRAME SHAPE (1200, 1920)
INFO:root:FRAME SHAPE (1200, 1920)
INFO:root:FRAME SHAPE (1200, 1920)
INFO:root:FRAME SHAPE (1200, 1920)
INFO:root:FRAME SHAPE (1200, 1920)


Num cycles:1800
Desired avg basler interframe usec:11111
Num elements:3
420,11531,22642,
3570,14681,25792,


INFO:root:FRAME SHAPE (576, 640)
INFO:root:FRAME SHAPE (576, 640)
INFO:root:FRAME SHAPE (576, 640)
INFO:root:FRAME SHAPE (576, 640)
recording progress (s):  17%|███████████████████████████████████▌                                                                                                                                                                                 | 10/60 [00:10<00:50,  1.00s/it]Process Writer-7:
Traceback (most recent call last):
  File "/home/dattalab/miniconda3/envs/multicam/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/home/dattalab/code/multicamera_acquisition/multicamera_acquisition/acquisition.py", line 241, in run
    self.append(img)
  File "/home/dattalab/code/multicamera_acquisition/multicamera_acquisition/acquisition.py", line 264, in append
    self.pipe = write_frame(
  File "/home/dattalab/code/multicamera_acquisition/multicamera_acquisition/video_io.py", line 98, in write_frame
    pipe.stdin.write(fr

BrokenPipeError: [Errno 32] Broken pipe

During handling of the above exception, another exception occurred:

OSError: [Errno 28] No space left on device

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/dattalab/miniconda3/envs/multicam/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/home/dattalab/code/multicamera_acquisition/multicamera_acquisition/acquisition.py", line 222, in run
    with open(self.metadata_file_name, "a") as metadata_f:
OSError: [Errno 28] No space left on device
recording progress (s):  77%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▎                                                 | 46/60 [00:46<00:14,  1.00s/it]

### Plot frames

In [ ]:
from pathlib2 import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.set_loglevel (level = 'warning')

In [ ]:
name = camera_list[0]['name']
df = pd.read_csv(list(video_out.glob(f'{name}*.csv'))[0])

In [ ]:
for cam in camera_list:
    name = cam['name']
    print(name)
    df = pd.read_csv(list(video_out.glob(f'{name}*.csv'))[0])
    fig, axs = plt.subplots(ncols = 1, nrows = 5, gridspec_kw= {'height_ratios':[1,1,1,1,1]}, figsize=(10,8))
    nrows = len(df)
    axs[0].set_title(f"{name}: frame diff")
    axs[0].plot(np.diff(df.frame_id.values))
    axs[0].set_xlim([0, len(df)])
    axs[1].set_title(f"{name}: camera timestamp diff")
    axs[1].plot(np.diff(df.frame_timestamp.values) / np.median(np.diff(df.frame_timestamp.values)))
    #axs[1].plot(np.diff(df.frame_timestamp.values))
    axs[1].set_xlim([0, len(df)])
    axs[2].set_title(f"{name}: computer timestamp (uid) diff")
    axs[2].plot(np.diff(df.frame_image_uid.values) / np.median(np.diff(df.frame_image_uid.values)))
    axs[2].set_xlim([0, len(df)])
    axs[3].set_title(f"{name}: queue size")
    axs[3].plot(df.queue_size.values)
    axs[3].set_xlim([0, len(df)])
    axs[3].set_xlabel('Frames')
    axs[4].hist(1/(np.diff(df.frame_timestamp.values)* 1e-9), bins=100);
    axs[4].set_xlabel('Framerate')
    plt.tight_layout()
    plt.show()
    
    time_elapsed = (df.frame_timestamp.values[-1] - df.frame_timestamp.values[0]) * 1e-9
    avg_framerate = np.mean(np.diff(df.frame_timestamp.values))
    print(time_elapsed)
    print(1 / (avg_framerate* 1e-9))

In [ ]:
from multicamera_acquisition.video_io import read_frames

In [ ]:
name = 'bottom'
filename = list(video_out.glob(f'{name}*.avi'))[0].as_posix()
frames = read_frames(
    filename, 
    pixel_format = "gray16",
    frames = list(range(294))
)
fig, axs = plt.subplots(figsize=(12, 4), ncols =2)
axs[0].imshow(frames[0], cmap = 'turbo')
axs[1].hist(frames[0].flatten())

In [ ]:
frames.shape

In [ ]:
for i, frame in enumerate(frames[:16]):
    plt.figure()
    plt.imshow(frame)
    ax = plt.gca()
    ax.set_title(i)

In [ ]:
plt.imshow(frames[17])

In [ ]:
filename = list(video_out.glob(f'{name}*.avi'))[0].as_posix()
frames = read_frames(
    filename, const unsigned int BASLER_IR_PULSE_WIDTH_
    pixel_format = "gray16",
    frames = [0,1,2]
                    
)
fig, axs = plt.subplots(figsize=(12, 4), ncols =2)
axs[0].imshow(frames[0], cmap='Greys_r')
axs[1].hist(frames[0].flatten())